In [61]:
from SafeTransformer import SafeTransformer
from sklearn.datasets import load_digits
import numpy as np
from sklearn.model_selection import train_test_split
data = load_digits()

In [62]:
import pandas as pd
X = pd.DataFrame(data.data)
y = pd.Series(data.target)

In [63]:
X = X.loc[((y==7) | (y==2) | (y==4)), :]
y = y.loc[((y==7) | (y==2) | (y==4))]

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [65]:
from sklearn.ensemble import GradientBoostingClassifier
surrogate_model = GradientBoostingClassifier().fit(X_train, y_train)

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
base_model = LogisticRegression().fit(X_train, y_train)
base_predictions = base_model.predict(X_test)

In [67]:
accs = []
pens = [1]
best_score = 0
best_pen = 0
for pen in pens:
    
    safe_transformer = SafeTransformer()

    safe_transformer = safe_transformer.fit(X_train, surrogate_model, penalty = pen)

    X_train_transformed = safe_transformer.transform(X_train)

    X_test_transformed = safe_transformer.transform(X_test)

    logistic_model_transformed = LogisticRegression()
    
    logistic_model_transformed = logistic_model_transformed.fit(X_train_transformed, y_train)

    surrogate_predictions = logistic_model_transformed.predict(X_test_transformed)
    acc = accuracy_score(y_test, surrogate_predictions)
    accs.append(acc)
    #print(mean_squared_error(y_test, standard_predictions), error)
    if acc > best_score:
        best_score = acc
        best_pen = pen

In [79]:
print(best_score)
print(accuracy_score(y_test, base_predictions))
print(accuracy_score(y_test, surrogate_model.predict(X_test)))

0.9185185185185185
1.0
0.9851851851851852


In [27]:
safe_transformer = SafeTransformer()

safe_transformer = safe_transformer.fit(X_train, surrogate_model, penalty = 0.0001)

X_train_transformed = safe_transformer.transform(X_train)

X_test_transformed = safe_transformer.transform(X_test)

In [28]:
X_train_transformed

,"A_(7.90, +Inf)","B_(4.40, +Inf)","C_(6.70, +Inf)","D_(2.50, +Inf)"
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [26]:
print(accs)

[0.9185185185185185]


In [83]:
X_test_transformed.shape

(135, 64)

In [82]:
X_test.shape

(135, 64)

In [85]:
X_test_transformed.apply(pd.value_counts)

,"0_(0.00, +Inf)","1_(5.00, +Inf)","2_(16.00, +Inf)","3_(16.00, +Inf)","4_(16.00, +Inf)","5_(16.00, +Inf)","6_(16.00, +Inf)","7_(14.00, +Inf)","8_(2.00, +Inf)","9_(15.00, +Inf)",...,"54_(16.00, +Inf)","55_(10.00, +Inf)","56_(1.00, +Inf)","57_(8.00, +Inf)","58_(16.00, +Inf)","59_(16.00, +Inf)","60_(16.00, +Inf)","61_(16.00, +Inf)","62_(16.00, +Inf)","63_(16.00, +Inf)"
0.0,NaN,133,132,98,95,119,133,135.0,135.0,135.0,...,133,135.0,135.0,134,125,102,107,123,130,135.0
1.0,135.0,2,3,37,40,16,2,NaN,NaN,NaN,...,2,NaN,NaN,1,10,33,28,12,5,NaN
